#### Upload functions

In [2]:
from bb2022_functions import *
%matplotlib inline

## Import and format metadata from lab, and BBMP

In [3]:
md = pd.read_csv("metadata_merged.csv")
merged = pd.read_csv("metadata_niskin.csv")
all_md = pd.read_csv("allmetadata.csv")

## Add prokaryotic community

Data prep

In [4]:
#generate a dataframe from all specified amplicon
df, comm = consolidate_tables('16S')

Found all 16S tables.
Successfully saved all tables.


In [5]:
merged = merge_metadata(df, all_md)

Set up metadata ...
Saved merged_asvs_metadata.tsv


In [6]:
separated = pick_metadata(merged)

Appended all taxonomies to taxos
Saved separated by metadata dataframe.


Remove chloroplast, cyanobacteria, mitochondria contaminants

In [7]:
searchfor = ["Cyanobacteria", "Chloroplast", "Mitochondria"]
contaminants = separated[separated.Taxon.str.contains('|'.join(searchfor))]
separated = separated[~separated.Taxon.str.contains('|'.join(searchfor))]
separated = separated.reset_index(drop=True)

Run the following code to generate "newseparated" which is the union of small and large size fractions

In [8]:
newseparated = make_defract(all_md, separated)

/Users/Diana/Documents/escuela/phd/size_fractions/bb2022_functions.py:187: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  sep_SLRA['Newfeature_frequency'] = sep_SLRA['feature_frequency'] * sep_SLRA['DNApr']
/Users/Diana/Documents/escuela/phd/size_fractions/bb2022_functions.py:188: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  sep_SLRA['Newff'] = sep_SLRA.groupby(['feature_id', 'weekn', 'depth'])['Newfeature_frequency'].transform('sum')
/Users/Diana/Documents/escuela/phd/size_fractions/bb2022_functions.py:192

### Richness analysis

In [ ]:
sfd1 = separated[['sampleid','size_code', 'weekn', 'nASVs', 'depth']].copy()

In [ ]:
#group the dataframe with all features to obtain either the mean or std of number of features per size fraction
sfd1.groupby(['size_code']).std()

In [ ]:
sfd1.describe()

In [ ]:
#run the visualisations for alpha diversity and run pairwise t-tests between size fractions for richness values
anova, results = boxplot_depth(separated, comm, 60, 'nASVs', 'Number of ASVs')

In [ ]:
results

## Simple regression analysis between pairs of size fractions

In [ ]:
#data prep
if comm == '16S':
    comm_id = '02-PROKs'
else:
    comm_id = '02-EUKs'

depth = 1

d1 = newseparated.loc[newseparated['depth'] == depth]
forpl = d1[['ratio', 'feature_id', 'sampleid', 'weekn', 'depth', 'size_code', 'Phylum', 'Family']].copy()
slwplot = forpl.pivot_table(index=["feature_id", "depth", 'weekn','Phylum', 'Family'], columns="size_code", values='ratio').fillna(0)
slwplot = slwplot.reset_index()

In [ ]:
q75, q25 = np.percentile(slwplot['L'], [75 ,25])
iqr = q75 - q25

In [ ]:
def outlier(df):
    new_df = df.copy()
    numeric_cols = ['L', 'S', 'SL', 'W']
    
    q1 = np.percentile(new_df[numeric_cols],25, axis=0)
    q3 = np.percentile(new_df[numeric_cols],75, axis=0)
    IQR = q3 - q1
    lower_limit = q1 - (1.5*IQR)
    upper_limit = q3 + (1.5*IQR)
    mask = (new_df[numeric_cols] < lower_limit) | (new_df[numeric_cols] > upper_limit)
    new_df[numeric_cols] = new_df[numeric_cols].mask(mask)
    return new_df

In [ ]:
newslw = outlier(slwplot)

In [ ]:
newslw

In [ ]:
newslw['L'].isna().sum()

In [ ]:
#rerun the regression by removing the 4 pathogens found in weeks 10 and 11
slwplot.sort_values('W')

In [ ]:
slwplot.drop([35,36,2353,2200], axis=0, inplace=True)

slwplot.sort_values('W')

In [ ]:
slwplot = slwplot.loc[slwplot['feature_id'] != '4cb5c79abe0a3b611671b1f356a86f19']
slwplot = slwplot.loc[slwplot['feature_id'] != '51d96e96a3350beedece3878d6d0b3e7']
slwplot = slwplot.loc[slwplot['feature_id'] != '015600cdcdfdc7333d251c9c9160eb72']

In [ ]:
Y = slwplot['S']
X = slwplot['W']

X = sm.add_constant(X)

#fit the model
model = sm.OLS(Y, X, missing='drop')
model_result = model.fit()
model_result.summary()

In [ ]:
sns.histplot(model_result.resid);

In [ ]:
mu, std = stats.norm.fit(model_result.resid)
mu, std

In [ ]:
fig, ax = plt.subplots()
# plot the residuals
sns.histplot(x=model_result.resid, ax=ax, stat="density", linewidth=0, kde=True)
ax.set(title="Distribution of residuals", xlabel="residual")

# plot corresponding normal curve
xmin, xmax = plt.xlim() # the maximum x values from the histogram above
x = np.linspace(xmin, xmax, 100) # generate some x values
p = stats.norm.pdf(x, mu, std) # calculate the y values for the normal curve
sns.lineplot(x=x, y=p, color="orange", ax=ax)
plt.show()

In [ ]:
sns.boxplot(x=model_result.resid, showmeans=True);

In [ ]:
sm.qqplot(model_result.resid, line='s');

In [ ]:
fig, ax = plt.subplots()
fig.set_size_inches(6, 5)

sns.set_style('white')
fig = sm.graphics.plot_fit(model_result,1, vlines=False, ax=ax)
ax.set_ylabel("Defractionated")
ax.set_xlabel("Whole")
ax.set_title("Fitted values linear regression")

plt.savefig('outputs/'+comm_id+'/asv_'+str(depth)+'_scatter_RL.png', dpi=200, bbox_inches="tight")

plt.show()

In [ ]:
model_result.fittedvalues

In [ ]:
Y_max = Y.max()
Y_min = Y.min()

plt.figure(figsize=(5, 4))


ax = sns.scatterplot(x=model_result.fittedvalues, y=Y)
ax.set(ylim=(Y_min, Y_max))
ax.set(xlim=(Y_min, Y_max))
ax.set_xlabel("Predicted value")
ax.set_ylabel("Observed value")

X_ref = Y_ref = np.linspace(Y_min, Y_max, 100)
plt.plot(X_ref, Y_ref, color='red', linewidth=1)

plt.savefig('outputs/'+comm_id+'/asv_'+str(depth)+'_scatterRL.png', dpi=200, bbox_inches="tight")

plt.show()

investigate temporal pattern thorugh all depths of outliers

In [ ]:
slwplot.sort_values('W')

In [ ]:
slwplot.loc[slwplot['feature_id'] == '75ceeaa937c64399438614ca3706cf2a']

In [ ]:
supsel = newseparated.loc[newseparated['feature_id'] == '75ceeaa937c64399438614ca3706cf2a'].sort_values('feature_frequency')
supsel['depth'].unique()

In [ ]:
supsel.sort_values('weekn')

In [ ]:
sns.scatterplot(data = supsel, x = 'weekn', y = 'ratio', hue='depth')

In [ ]:
#draw interactive plotly to identify outliers
import plotly.express as px

df = px.data.tips()
fig = px.scatter(slwplot, x="W", y="SL", color="weekn", trendline="ols")
fig.show()

results = px.get_trendline_results(fig)
print(results)

#results.query("weekn ==  and Phylum == ").px_fit_results.iloc[0].summary()

### Calculate log2 fold change per feature at the phylum level of abundance between size fractions to identify which taxonomic group is driving the simple regression off x=y

In [ ]:
#add pseudo count for log-calculations and zero divisions
slwplot['SL'] = slwplot['SL'] + 0.0000001
slwplot['W'] = slwplot['W'] + 0.0000001

#calculate log2 fold change
slwplot['OR'] = (slwplot['W'] - slwplot['SL']) / slwplot['SL']
slwplot['fold_change'] = slwplot['W']/slwplot['SL']
slwplot['log2_fold_change'] = np.log2(slwplot['fold_change'])

In [ ]:
#make dual plot of log2FC and mean relative abundances side by side including error bars in the plots
data = slwplot[['log2_fold_change','Phylum', 'W', 'SL']].copy()

data['Phylum'] = data['Phylum'].map(lambda x: x.lstrip('p__'))

# Group by index labels and take the means and standard deviations for each group
data['avg_W'] = data['W'].groupby(data['Phylum']).transform('mean')
data['std_W'] = data['W'].groupby(data['Phylum']).transform('std')
data['avg_SL'] = data['SL'].groupby(data['Phylum']).transform('mean')
data['std_SL'] = data['SL'].groupby(data['Phylum']).transform('std')
data['means'] = data['log2_fold_change'].groupby(data['Phylum']).transform('mean')
data['stds'] = data['log2_fold_change'].groupby(data['Phylum']).transform('std')

data['positive'] = data['means'] > 0

fig, axes = plt.subplots(ncols=2, sharey=True, figsize=(8, 10))

axes[0].barh(data['Phylum'], data['means'],
         xerr = data['stds'],
         error_kw=dict(lw=0.5, capsize=1, capthick=0.5),
         color=data.positive.map({True: 'g', False: 'r'}))

axes[1].barh(data['Phylum'], data['avg_W'],
            xerr = data['std_W'],
         error_kw=dict(lw=0.5, capsize=1, capthick=0.5))

#axes[2].barh(data['Phylum'], data['avg_SL'],
#            xerr = data['std_SL'],
#         error_kw=dict(lw=0.5, capsize=1, capthick=0.5))

plt.gca().invert_yaxis()


plt.savefig('outputs/'+comm_id+'/log2foldchange_d'+str(depth)+'.png', bbox_inches='tight', dpi=300)

#plt.show()

In [ ]:
#bar plot of log2FC per phylum without error bars
data = slwplot[['log2_fold_change','Phylum']].copy()
data['Phylum'] = data['Phylum'].map(lambda x: x.lstrip('p__'))

# Group by index labels and take the means and standard deviations for each group
#data['means'] = data['log2_fold_change'].groupby(data['Phylum']).transform('mean')
#data['stds'] = data['log2_fold_change'].groupby(data['Phylum']).transform('std')

data['positive'] = data['log2_fold_change'] > 0

plt.figure(figsize=(6,11))
plt.barh(data['Phylum'], data['log2_fold_change'],
         color=data.positive.map({True: 'g', False: 'r'}))
plt.gca().invert_yaxis()

plt.savefig('outputs/'+comm_id+'/log2foldchange_d'+str(depth)+'_noerr.png', bbox_inches='tight', dpi=300)

#plt.show()

In [ ]:
not_in_W = slwplot.loc[slwplot['W'] == 0]
#not_in_W['feature_id'].nunique()
not_in_W_from_S = not_in_W.loc[not_in_W['L'] == 0]
#not_in_W_from_S['feature_id'].nunique()

#calculate the percentage of features coming from the small size fraction that are absent from the whole  
not_in_W_from_S['feature_id'].nunique()/not_in_W['feature_id'].nunique()

In [ ]:
sns.scatterplot(data=not_in_W, x='S', y='SL', hue='weekn')

### run lmplot for each pair of sets

In [ ]:
#loglog plot
depths = [1,5,10,30,60]

for depth in depths:
    d1 = newseparated.loc[newseparated['depth'] == depth]
    forpl = d1[['ratio', 'feature_id', 'sampleid', 'weekn', 'depth', 'size_code', 'Phylum']].copy()
    slwplot = forpl.pivot_table(index=["feature_id", "depth", 'weekn', 'Phylum'], columns="size_code", values='ratio').fillna(0)
    slwplot = slwplot.reset_index()
    sns.set_style("white")

    #slwplot = slwplot.loc[slwplot['weekn'] = 10]
    #slwplot = slwplot.loc[slwplot['weekn'] = 11]

    slwplot = slwplot.rename(columns={"depth": "Depth"})
    slwplot["weekn"] = pd.to_numeric(slwplot["weekn"])
    g = sns.scatterplot(x="W", y="SL", data=slwplot, palette=['black'])#, hue='Phylum', alpha=0.6) #, hue="weekn");
    
    #uncomment for log-log
    #ax.set(xscale="log", yscale="log")
    g.set_ylabel("Defractionated",fontsize=15)
    g.set_xlabel("Unfractionated",fontsize=15)
    g.tick_params(labelsize=12)
    #g.legend(loc='center left', bbox_to_anchor=(1.25, 0.5), ncol=3)
    plt.legend([],[], frameon=False)
    
    #plt.savefig('outputs/lmplot_'+comm+str(depth)+'WSL.png', bbox_inches='tight', dpi=300)

    plt.show()

### Phylogenetic analysis: taxonomic bar plots of relative abundance per depth and size fraction

In [ ]:
#get a list of top taxa to provide the palette for the visualisation
toptaxa = newseparated[['feature_id', 'feature_frequency', 'Taxon', 'size_code', 'depth','weekn', 'Phylum']].copy()
toptaxa = toptaxa.drop_duplicates()
df_agg = toptaxa.groupby(['size_code','Phylum', 'depth']).agg({'feature_frequency':sum})
topd = df_agg['feature_frequency'].groupby(['size_code', 'depth'], group_keys=False).nlargest(10)
topd = topd.to_frame()
topd = topd.reset_index()
listoftop = topd['Phylum'].unique()

#set a palette for the toptaxa
hex_colors_dic = {}
rgb_colors_dic = {}
hex_colors_only = []
for name, hex in matplotlib.colors.cnames.items():
    hex_colors_only.append(hex)
    hex_colors_dic[name] = hex
    rgb_colors_dic[name] = matplotlib.colors.to_rgb(hex)
    
palette_dict = {taxon: color for taxon, color in zip(listoftop, px.colors.sequential.Plasma)}

In [ ]:
phyld, top10d = taxbarplot(newseparated, 'Phylum', 1, 5)

#### Top taxon longitudinal analysis

In [ ]:
# newseparated.loc[(newseparated['weekn']==12)&
#                  (newseparated['depth']==10)]
#                & (newseparated['size_code']=='W')]

In [ ]:
sfd=newseparated
toptaxa = sfd[['feature_id', 'feature_frequency', 'Taxon', 'size_code', 'depth','weekn', 'Class']].copy()
toptaxa = toptaxa.drop_duplicates()
df_agg = toptaxa.groupby(['size_code','Class', 'depth', 'weekn']).agg({'feature_frequency':sum})
topd = df_agg['feature_frequency'].groupby(['size_code', 'depth','weekn'], group_keys=False).nlargest(1)
topd = topd.to_frame()
topd = topd.reset_index()

In [ ]:
topd.loc[topd['weekn'] == 12]

In [ ]:
#list the unique top taxa
topd['Class'].unique()

In [ ]:
type_dic = {
    #"c__Cyanobacteriia": 1,
    "c__OM190": 3,
    "c__Bacteroidia": 2,
    "c__Gammaproteobacteria": 5,
    "c__Alphaproteobacteria": 4}

In [ ]:
#make a season column
topd['comm_type'] = ''

for tx, ctype in type_dic.items():
    topd.loc[topd['Class'] == tx, 'comm_type'] = ctype
    
topd

In [ ]:
topd["sc_weekn"] = topd["depth"].astype(str) + topd["size_code"]
topd

In [ ]:
topd = topd.sort_values(['depth', 'size_code'])

In [ ]:
topd.loc[topd['sc_weekn'] == '10W']

In [ ]:
topd

In [ ]:
topdlist = topd['sc_weekn'].tolist()

def uniqlist(seq):
    seen = set()
    seen_add = seen.add
    return [x for x in seq if not (x in seen or seen_add(x))]

mylist = uniqlist(topdlist)

In [ ]:
glue = topd.pivot(index="sc_weekn", columns="weekn", values="comm_type")
glue = glue.reindex(mylist)
glue = glue[glue.columns].astype(float)

In [ ]:
from matplotlib.colors import ListedColormap
cmap_dict = {#1: '#77AADD',
             2: '#EEDD88', 3: '#99DDFF', 4: '#BBCC33', 5:'#DDDDDD'}
cmap = ListedColormap([cmap_dict[i] for i in range(2,6,1)])

In [ ]:
sns.set_style('ticks')
plt.figure(figsize=(5, 5))


ax = sns.heatmap(glue, fmt='f', yticklabels=True, linewidths=.5, cmap=cmap)
ax.axhline(4, ls='--')
ax.axhline(8, ls='--')
ax.axhline(12, ls='--')
ax.axhline(16, ls='--')

ax.set_xticks(range(1, 16, 4))

plt.savefig('outputs/heatmap_'+comm+'top1clasno_chloroplast.png', bbox_inches='tight', dpi=300)

plt.show()

#### Longitudinal analysis of top 3 taxa

Run these lines only for the 18S rRNA analysis

In [ ]:
#18S has many more classes as top values to
top518s = topd['Class'].value_counts()[:4].index.tolist()

In [ ]:
topd.loc[~topd["Class"].isin(top518s), "Class"] = "Other"

In [ ]:
newdf = topd.groupby(['size_code', 'depth','weekn'])['Class'].apply(lambda x: list(set(x)))
newdf = newdf.reset_index()

In [ ]:
result = newdf.copy()

In [ ]:
result = newdf.Class.sort_values().apply(lambda x: sorted(x))
result = pd.DataFrame(result).reset_index(drop=True)

In [ ]:
result.Class.apply(tuple).unique()

In [ ]:
topd['Class'].unique()

In [ ]:
type_dic = {
    'c__Cryptophyceae':1, 'c__Dinophyceae':2, 'c__Prymnesiophyceae':3,
       'c__Mediophyceae':4, 'c__Monogononta':5, 'c__Tentaculata':6,
       'c__Maxillopoda':7, 'c__Thecofilosea':8, 'Unassigned':9, 'c__Insecta':10,
       'c__MAST-2':11, 'c__Hydrozoa':12, 'c__Syndiniales':13,
       'c__Intramacronucleata':14, 'c__Pucciniomycetes':15,
       'c__Mamiellophyceae':16, 'c__Raphidophyceae':17, 'c__MAST-1A':18,
       'c__Bicosoecida':19, 'c__Polychaeta':20, 'c__Tremellomycetes':21,
       'c__Embryophyta':22, 'c__Dothideomycetes':23, 'c__Incertae_Sedis':24,
       'c__MAST-7A':25
}

In [ ]:
type_dic = {'Other':5, 'c__Dinophyceae':1, 'c__Mediophyceae':2,
       'c__Intramacronucleata':3, 'c__Syndiniales':4}

In [ ]:
result['liststring'] = result['Class'].apply(lambda x: ','.join(map(str, x)))

In [ ]:
result['liststring'].unique()

In [ ]:
#use this dic if looking at community type by 3 top taxa
type_dic = {'c__Bacteroidia,c__Cyanobacteriia,c__OM190':1,
       'c__Gammaproteobacteria,c__Bacteroidia,c__Cyanobacteriia':2,
       'c__Alphaproteobacteria,c__Bacteroidia,c__Cyanobacteriia':3,
       'c__Bacteroidia,c__Cyanobacteriia,c__Planctomycetes':4,
       'c__Bacteroidia,c__Alphaproteobacteria,c__OM190':5,
       'c__Bacteroidia,c__OM190,c__Planctomycetes':6,
       'c__Gammaproteobacteria,c__Bacteroidia,c__OM190':7,
       'c__Gammaproteobacteria,c__Bacteroidia,c__Planctomycetes':8,
       'c__Cyanobacteriia,c__OM190,c__Planctomycetes':9,
       'c__Alphaproteobacteria,c__Gammaproteobacteria,c__Cyanobacteriia':10,
       'c__Gammaproteobacteria,c__Bacteroidia,c__Alphaproteobacteria':11,
       'c__Bacteroidia,c__Alphaproteobacteria,c__Cyanobacteriia':12,
       'c__Nitrososphaeria,c__Gammaproteobacteria,c__Bacteroidia':13,
       'c__Gammaproteobacteria,c__Alphaproteobacteria':14,
       'c__Cyanobacteriia':15}

In [ ]:
#make a season column
topd['comm_type'] = ''

for tx, ctype in type_dic.items():
    topd.loc[topd['Class'] == tx, 'comm_type'] = ctype
    
topd

In [ ]:
result['comm_type'] = ''

for tx, ctype in type_dic.items():
    result.loc[result['liststring'] == tx, 'comm_type'] = ctype
    
result

In [ ]:
topd = result.copy()

In [ ]:
topd["sc_weekn"] = topd["depth"].astype(str) + topd["size_code"]
topd

In [ ]:
topd = topd.sort_values(['depth', 'size_code'])

In [ ]:
topdlist = topd['sc_weekn'].tolist()

def uniqlist(seq):
    seen = set()
    seen_add = seen.add
    return [x for x in seq if not (x in seen or seen_add(x))]

mylist = uniqlist(topdlist)

In [ ]:
glue = topd.pivot(index="sc_weekn", columns="weekn", values="comm_type")
glue = glue.reindex(mylist)
glue = glue[glue.columns].astype(float)

In [ ]:
 '#77AADD', '#EE8866', '#EEDD88', '#FFAABB', '#99DDFF', '#44BB99', '#BBCC33', '#AAAA00', '#DDDDDD'.

In [ ]:
from matplotlib.colors import ListedColormap
cmap_dict = {1: '#77AADD', 2: '#EEDD88', 3: '#99DDFF', 4: '#BBCC33', 5:'#DDDDDD'}
cmap = ListedColormap([cmap_dict[i] for i in range(1,6,1)])

In [ ]:
cmap_dict ={1:'#125A56', 2:'#00767B', 3:'#238F9D', 4:'#42A7C6', 5:'#60BCE9',
            6:'#9DCCEF', 7:'#C6DBED', 8:'#DEE6E7', 9:'#ECEADA', 10:'#F0E6B2',
            11:'#F9D576', 12:'#FFB954', 13:'#FD9A44', 14:'#F57634', 15:'#E94C1F'}#, '#D11807', '#A01813'.
cmap = ListedColormap([cmap_dict[i] for i in range(1,16,1)])

In [ ]:
sns.set_style('ticks')
plt.figure(figsize=(5, 5))


ax = sns.heatmap(glue, fmt='f', yticklabels=True, linewidths=.5, cmap=cmap)
ax.axhline(4, ls='--')
ax.axhline(8, ls='--')
ax.axhline(12, ls='--')
ax.axhline(16, ls='--')

ax.set_xticks(range(1, 16, 4))


plt.savefig('outputs/heatmap_'+comm+'top1class_reducde.png', bbox_inches='tight', dpi=300)

plt.show()

In [ ]:
    sfd=separated[separated.depth==depth]
    toptaxa = sfd[['feature_id', 'feature_frequency', 'Taxon', 'size_code', 'depth','weekn', level]].copy()
    toptaxa = toptaxa.drop_duplicates()
    df_agg = toptaxa.groupby(['size_code',level, 'depth']).agg({'feature_frequency':sum})
    topd = df_agg['feature_frequency'].groupby('size_code', group_keys=False).nlargest(topn)
    topd = topd.to_frame()
    topd = topd.reset_index()


    df_agg = df_agg.reset_index()
    df_agg['set_name'] = df_agg['size_code']+df_agg['depth'].astype(str)
    
    cumulab = separated[['feature_frequency', 'depth', 'size_code', 'Genus']].copy()
    cumulab1 = cumulab.groupby(['Genus']).agg({'feature_frequency':sum})

    resultpivot = df_agg.pivot_table(index=level, columns='set_name', values='feature_frequency')
    resultpivot = resultpivot.fillna(0)
    resultpivot[resultpivot != 0] = 1
    tosave = pd.merge(resultpivot, cumulab1, left_index=True, right_index=True)
    tosave.to_csv(level+'_'+str(depth)+'16S_relab.csv')
    
    top10d_list = topd[level].unique()
    top10d = sfd.copy()
    top10d.loc[~top10d[level].isin(top10d_list), level] = 'Other' #isnot in top list
    phyld = top10d.groupby(['size_code','weekn', level])['ratio'].sum()
    phyld = phyld.reset_index()


    fig = px.bar(phyld, x="size_code", y="ratio", facet_col="weekn", color=level, labels={
                     "feature_frequency": "Relative abundance",
                     "size_code": "",
                     "weekn": "w"}, color_discrete_map=palette_dict)
    fig.update_xaxes(type='category', dtick=1)
    fig.update_layout(
        title="Relative abundance of top 10" + level + 'observed at Depth' + str(depth),
        yaxis_title="Relative abundance",
        xaxis_title="Size fraction",
        legend_title=level,
        font=dict(size=8)
    )

    fig.show()
    #fig.write_image("outputs/fig1.png")
    #fig.to_image(format="png")

In [ ]:
phyld, top10d = taxbarplot(newseparated, 'Class', 5, 5)

In [ ]:
plot_df2 = plot_df2.drop_duplicates()

In [ ]:
plot_df3 = plot_df2.set_index('sampleid')
plot_df3

In [ ]:
permanova2 = permanova(dm, plot_df3, 'Size code')
permanova2

In [ ]:
distance_matrix2 = distance_matrix.reset_index()
idedup = distance_matrix2['samples'].to_list()
dm = DistanceMatrix(distance_matrix, ids=idedup)
df123 = dm.to_data_frame()

In [ ]:
df123.to_csv('distance_matrix_5m16s.tsv', sep='\t')

In [ ]:
plot_df2.to_csv('METADATAtiny.txt', sep='\t')

In [ ]:
pca, pca_features, sfdclr = pcaplot(newseparated, 5, comm, 'Size code', 'DFr')

In [ ]:
newmetadata = newseparated[['sampleid', 'weekn', 'size_code', 'depth', 'depth_code', 'month_name']].copy()
newmetadata = newmetadata.drop_duplicates()
newmetadata.to_csv('METADATA.tsv', sep='\t')

In [ ]:
pca_features

In [ ]:
from skbio.stats.distance import permanova

In [ ]:
level = 'feature_id'
if level == 'feature_id':
    id = 'ASV'
else:
    id = level

subtitile = 'subtitle'

In [ ]:
dfplot, level = calcperc(comm, separated, level)
# variables
labels = ['S ∩ W', 'L ∩ W','W ∩ (S ∩ L)']
colors = ['#1D2F6F', '#8390FA', '#C49CD3']
title = 'Weighted proportion of shared '+id+' between size fractionated and non size fractionated samples'

plot_stackedbar_p(dfplot, labels, colors, title, subtitle, level)

In [ ]:
dfplot, level, dfplot_unweighted = calcperc_defrac_unweighted(comm, newseparated, level)
# variables
labels = ['Size fractionated samples', 'Not size fractionated samples','Both']
colors = ['#1D2F6F', '#8390FA', '#6EAF46']
title = 'Unweighted proportion of shared '+id+' between size fractionated and non size fractionated samples'

plot_stackedbar_p(dfplot, labels, colors, title, subtitle, level)

In [ ]:
dfplot, level = calcperc_defrac(comm, newseparated, level)
# variables
labels = ['SF samples', 'NSF samples','Both', 'DFr']
colors = ['#1D2F6F', '#8390FA', '#6EAF46', '#de282e']
title = 'Weighted proportion of shared '+id+' between size fractionated and non size fractionated samples'

plot_stackedbar_p(dfplot, labels, colors, title, subtitle, level)

In [ ]:
dfplotSLNSF, dfplot_normalized, level = calcperc_SLNSF(comm, separated, level)
# variables
labels = ['S', 'L','S ∩ L', 'W']
colors = ['#976BE5','#E56BE5','#FF96FF', '#6EAF46']
title = 'Weighted proportion of shared '+ id +' between size fractionated and non size fractionated samples'

plot_stackedbar_p_SLNSF(dfplotSLNSF, labels, colors, title, subtitle, level, 30.1, 5)

In [ ]:
dfplotLSW, dfplot_normalized, level = calcperc_LSW(comm, separated, level)
#variables
labels = ['NSF samples', 'Large and whole','Small and whole', 'Large, small, and whole']
colors = ['#8390FA', '#FF0000', '#DAD746', '#E89618']
title = 'Weighted proportion of shared '+ id +' between size fractionated and non size fractionated samples'

plot_stackedbar_p_SLNSF(dfplotLSW, labels, colors, title, subtitle, level)

In [ ]:
dfplotLS_W, dfplot_normalized, level = calcperc_LS_W(comm, separated, level)
#variables
labels = ['NSF samples', 'Large and whole','Small and whole']
colors = ['#8390FA', '#FF0000', '#DAD746']
title = 'Weighted proportion of shared '+ id +' between size fractionated and non size fractionated samples'

plot_stackedbar_p_SLNSF(dfplotLS_W, labels, colors, title, subtitle, level, 45.1, 5)

In [ ]:
newseparated

In [ ]:
level = 'feature_id'

toptaxa = separated[[level, 'feature_frequency', 'Taxon', 'size_code', 'weekn']].copy()
    
toptaxa = toptaxa.drop_duplicates()
df_agg = toptaxa.groupby(['size_code',level]).agg({'feature_frequency':sum})
    
df_agg = df_agg.reset_index()
resultpivot = df_agg.pivot_table(index=level, columns='size_code', values='feature_frequency')
resultpivot = resultpivot.fillna(0)
    
df1 = resultpivot.copy()
    
df = resultpivot[['L', 'S', 'W']].copy()
Sonly = df[(df['L'] == 0) & (df['W'] == 0)]
Wonly = df[(df['L'] == 0) & (df['S'] == 0)]
Lonly = df[(df['S'] == 0) & (df['W'] == 0)]
LW = df[(df['S'] == 0) & (df['W'] != 0) & (df['L'] != 0)]
LS = df[(df['W'] == 0) & (df['S'] != 0) & (df['L'] != 0)]
SW = df[(df['W'] != 0) & (df['S'] != 0) & (df['L'] == 0)]
LSW = df[~(df == 0).any(axis=1)]
        
c = venn3(subsets = (len(Lonly), len(Sonly), len(LS), len(Wonly), len(LW), len(SW), len(LSW)), 
          set_labels = ('Large >3μm', 'Small 3-02μm', 'Whole water <0.22μm'),
          #set_colors=('#E56BE5', '#976BE5', '#6EAF46'),
          set_colors=('#E56BE5', '#976BE5', '#6EAF46'), alpha = 1);


plt.savefig("outputs/02-EUKs/D"+level+"_vennall.png", dpi=200, bbox_inches="tight")

** another idea is to run ancom of sizefraction specific and compare after the categories (run ancom on ?time or month.. or some other column) and compare the number/taxonomy of differentially abundant taxa recovered;
are we recovering the same diff ab taxa between the (1.SF samples, 2. NSF samples)

In [ ]:
pca, pca_features, sfdclr, dm, plot_df2, df, distance_matrix = pcaplot(newseparated, 1, comm, 'Size code')

## 16S ANCOM PER DEPTH

In [ ]:
newseparated.loc[newseparated['feature_id'] == '51d96e96a3350beedece3878d6d0b3e7']

In [ ]:
df1 = newseparated[newseparated.size_code != 'S']
df1

In [ ]:
df1 = df1[df1.size_code != 'L']
df1

In [ ]:
newseparated = df[df.size_code != 'S']

In [ ]:
newseparated = newseparated[newseparated.depth == 1]

In [ ]:
newbiom = newseparated.pivot_table(index="feature_id", columns="sampleid", values="feature_frequency")

In [ ]:
newbiom = newbiom.fillna(0)

In [ ]:
newbiom.to_csv('newbiomdepth1.tsv', sep="\t") 

In [ ]:
pca1_16, pca_features1_16, sfdclr1_16 = pcaplot(df1, 1, '16S', 'Size code')
DARejected_SC_taxonomy1_16, prcentile1_16 = run_ancom(df1, sfdclr1_16, 1, 'weekn')

In [ ]:
prcentile1_16 = prcentile1_16.reset_index()

In [ ]:
prcentile1_16.loc[prcentile1_16['feature_id'] == '80c73848b68ff95bd030fccfce011294']

In [ ]:
DARejected_SC_taxonomy1_16.sort_values(by='W')

? run a clustermap of top10 taxa of each deapth and color rows by depth, month, size code

In [ ]:
pca, pca_features, sfdclr = pcaplot(separated, 1, '16S')
DARejected_month_taxonomy_16_1, prcentile = run_ancom(sfdclr, 1, 'Month')

In [ ]:
plot_per_fid('16S', separated, 60, '50a0c3221c68046dfc96e032aff1ccd8')

In [ ]:
plot_df2.sort_values('dim1')

Permanova results from R into boxplots

In [ ]:
permresu = pd.read_csv('R_results/post_hoc_results.csv')

In [ ]:
permresu["depth_pairs"] = permresu["depth"].astype(str) + permresu["pairs"]

In [ ]:
ax = sns.catplot(
    permresu, kind="bar",
    x="p.adjusted", y="pairs", col="comm", hue="depth",
    height=4, aspect=1.3, palette="Greys", log=True
)
#ax.set(xlim=(0, 0.10))

ax.refline(x=0.05, color='red')

plt.savefig('outputs/perm_pvalues_logged.png', bbox_inches='tight', dpi=300)

plt.show()

In [ ]:
ax = sns.catplot(
    permresu, kind="bar",
    x="p.adjusted", y="pairs", col="comm", hue="depth",
    height=4, aspect=1.3, palette="Greys", log=True
)
#ax.set(xlim=(0, 0.10))

ax.refline(x=0.05, color='red')

plt.savefig('outputs/perm_pvalues_logged.png', bbox_inches='tight', dpi=300)

plt.show()